In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
from SAC_Distillation.DistilledSACAgent import DistilledSAC
from SAC_Distillation.Trajectories import SAC_ExperienceBuffer
from Hyperparameters import HYPERPARAMS as params
import numpy as np
import torch
import wandb

In [2]:
wandb.init(project="SAC_Distillation", entity="fede-")
wandb.config.update(params['sac_distilled'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.config.update({"device": device})

wandb: Currently logged in as: rullofederico16 (fede-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
def relocate_agents(env):
    return list(env.agents)  # simplified

# New helper to extract observation data for an agent
def get_agent_obs(obs, agent):
    agent_data = obs[agent]
    return np.array(agent_data[1]), np.array(agent_data[2])

In [4]:
env = UE(file_name="Env/DroneFlightv1", seed=1, no_graphics_monitor=True)
env = UPZBE(env)

In [5]:
agents = relocate_agents(env)
print(agents)

['Drone?team=0?agent_id=0', 'Drone?team=0?agent_id=0', 'Drone?team=0?agent_id=1', 'Drone?team=0?agent_id=1', 'Drone?team=0?agent_id=10', 'Drone?team=0?agent_id=10', 'Drone?team=0?agent_id=11', 'Drone?team=0?agent_id=11', 'Drone?team=0?agent_id=2', 'Drone?team=0?agent_id=2', 'Drone?team=0?agent_id=3', 'Drone?team=0?agent_id=3', 'Drone?team=0?agent_id=4', 'Drone?team=0?agent_id=4', 'Drone?team=0?agent_id=5', 'Drone?team=0?agent_id=5', 'Drone?team=0?agent_id=6', 'Drone?team=0?agent_id=6', 'Drone?team=0?agent_id=7', 'Drone?team=0?agent_id=7', 'Drone?team=0?agent_id=8', 'Drone?team=0?agent_id=8', 'Drone?team=0?agent_id=9', 'Drone?team=0?agent_id=9']


In [6]:
Buffer = SAC_ExperienceBuffer(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape,env.action_space(agents[0]).shape, params['sac_distilled'])

In [7]:
brain = DistilledSAC(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape,len(agents), params['sac_distilled'])

In [8]:
# for s in range(1, params['sac_distilled'].seed_episodes):
#     obs, done, t = env.reset(), [False for _ in env.agents], 0
#     while not all(done) or t < params['sac_distilled'].n_steps_random_exploration:
#         actions = {}
#         log_probs = {}
#         values = {}
#         agents = relocate_agents(env)
#         for agent in agents:
#             actions[agent] = env.action_space(agent).sample()
#             if agent not in obs.keys():
#                 continue
#             obs1, obs2 = get_agent_obs(obs, agent)
#             v1,v2 = brain.get_values(obs1,obs2, actions[agent],t)
#             values[agent] = torch.min(v1,v2)

#         actions = {agent: act for agent, act in actions.items() if agent in set(env.agents)}
#         next_obs, reward, done, _ = env.step(actions)

#         for agent in agents:
#             if agent not in next_obs.keys():
#                 continue
#             next_obs1, next_obs2 = get_agent_obs(next_obs, agent)
#             Buffer.store(obs1, obs2, actions[agent], reward[agent], next_obs1, next_obs2, done[agent])
#         obs = next_obs
#         done = [done[agent] for agent in agents if agent in done.keys()]
#         t += 1
#     print(f'Finished episode {s}')

# # Buffer.compute_advantages()
# print("Finished Rnd Exploration")
# env.close()

In [9]:
# Buffer.save('Collected_Experience/init_experience.npz')

In [10]:
# Buffer.load('Collected_Experience/init_experience.npz')
# Buffer.size=len(Buffer.camera_obs)
# Buffer.ptr=Buffer.size
# print(len(Buffer))

In [11]:
# brain.fine_tune_teacher(Buffer, epochs=params['sac_distilled'].train_epochs)

In [12]:
# brain.teacher.save("SavedModels/Teacher")

In [13]:
# brain.train(Buffer,step = params['sac_distilled'].seed_episodes*params['sac_distilled'].n_steps_random_exploration)

In [14]:
# Buffer._clear_old()

In [15]:
# brain.save("SavedModels/SAC_distilled_checkpoint.pth")
# print("Checkpoint saved successfully.")
# env.close()

In [16]:
# brain.load("SavedModels/SAC_distilled_checkpoint.pth")
# print("Checkpoint loaded successfully.")

In [17]:
# env = UE(file_name="Env/DroneFlightv1", seed=1, no_graphics_monitor=True)
# env = UPZBE(env)
# env.reset()
# agents = relocate_agents(env)

In [19]:
steps=0
obs = env.reset()
mean_reward = -np.inf  # or 0
while steps < params['sac_distilled'].max_steps:
    # obs = env.reset()
    done = [False for _ in env.agents]
    episode_rewards, t = [], 0
    while not all(done) or t < params['sac_distilled'].n_steps:
        actions, values = {}, {}
        agents = relocate_agents(env)

        for agent in agents:
            if agent not in obs.keys():
                continue

            camera_obs, pos_obs = get_agent_obs(obs, agent)
            action, _ = brain.get_action(camera_obs, pos_obs, train=True)

            v1,v2 = brain.get_values(camera_obs, pos_obs, action, steps+t)
            value = torch.min(v1,v2)

            actions[agent] = action.cpu().detach().numpy().flatten().tolist()
            values[agent] = value
        next_obs, reward, done, _ = env.step(actions)

        for agent in agents:
            if agent not in next_obs.keys():
                continue
            camera_obs, pos_obs = get_agent_obs(obs, agent)
            next_camera_obs, next_pos_obs = get_agent_obs(next_obs, agent)
            reward[agent] = reward[agent] * 10
            Buffer.store(camera_obs, pos_obs, actions[agent], reward[agent], next_camera_obs, next_pos_obs, done[agent])

        obs = next_obs
        t += 1
        episode_rewards.append(list(reward.values()))

    # Compute reward properly
    episode_flat_rewards = [r for r_list in episode_rewards for r in r_list]
    current_reward = np.mean(episode_flat_rewards)

    if current_reward > mean_reward:
        brain.save("SavedModels/SAC_distilled_trained.pth")
        print("Checkpoint saved successfully.")
        mean_reward = current_reward

    steps += t

    # Training step
    actor_loss, critic_loss, entropy_loss, distill_loss = brain.train(Buffer, steps)
    mean_val = np.mean([values[agent].mean().item() for agent in env.agents])
    if critic_loss > 1e6:
        brain.load("SavedModels/SAC_distilled_trained.pth")
        print("Checkpoint loaded successfully.")

    wandb.log({
        "Mean Reward": current_reward,
        "Steps": steps,
        "Actor Loss": actor_loss,
        "Critic Loss": critic_loss,
        "Entropy Loss": entropy_loss,
        "Distill Loss": distill_loss,
        "value": mean_val
    })

env.close()

Checkpoint saved successfully.


KeyboardInterrupt: 